<a href="https://colab.research.google.com/github/HLaw18/4105/blob/main/HW6_4105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

Mounted at /content/drive


In [3]:
#1a
file_path = '/content/drive/My Drive/Colab Notebooks/DataSets/Housing.csv'
data = pd.DataFrame(pd.read_csv(file_path))

X = data[['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement',
          'hotwaterheating', 'airconditioning', 'parking', 'prefarea']].values

data = pd.get_dummies(data, columns=['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea'], drop_first=True)

X = data[['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad_yes', 'guestroom_yes', 'basement_yes',
          'hotwaterheating_yes', 'airconditioning_yes', 'parking', 'prefarea_yes']].values
y = data['price'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train).view(-1, 1)
X_val = torch.FloatTensor(X_val)
y_val = torch.FloatTensor(y_val).view(-1, 1)

linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train.numpy(), y_train.numpy())
linear_reg_val_predictions = linear_reg_model.predict(X_val.numpy())

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X_train.shape[1]
hidden_size = 32
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    val_outputs = model(X_val)
    val_loss = criterion(val_outputs, y_val)

print(f'Validation Loss (Neural Network): {val_loss.item():.4f}')
num_parameters_nn = sum(p.numel() for p in model.parameters())
print(f'Number of parameters in the Neural Network: {num_parameters_nn}')

Epoch [100/5000], Loss: 25234774163456.0000
Epoch [200/5000], Loss: 25234738511872.0000
Epoch [300/5000], Loss: 25234665111552.0000
Epoch [400/5000], Loss: 25234547671040.0000
Epoch [500/5000], Loss: 25234386190336.0000
Epoch [600/5000], Loss: 25234172280832.0000
Epoch [700/5000], Loss: 25233908039680.0000
Epoch [800/5000], Loss: 25233597661184.0000
Epoch [900/5000], Loss: 25233241145344.0000
Epoch [1000/5000], Loss: 25232838492160.0000
Epoch [1100/5000], Loss: 25232404381696.0000
Epoch [1200/5000], Loss: 25231930425344.0000
Epoch [1300/5000], Loss: 25231418720256.0000
Epoch [1400/5000], Loss: 25230871363584.0000
Epoch [1500/5000], Loss: 25230292549632.0000
Epoch [1600/5000], Loss: 25229682278400.0000
Epoch [1700/5000], Loss: 25229036355584.0000
Epoch [1800/5000], Loss: 25228358975488.0000
Epoch [1900/5000], Loss: 25227660623872.0000
Epoch [2000/5000], Loss: 25226922426368.0000
Epoch [2100/5000], Loss: 25226152771584.0000
Epoch [2200/5000], Loss: 25225351659520.0000
Epoch [2300/5000], 

In [7]:
#1b
class ComplexNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(ComplexNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_sizes[2], output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

hidden_sizes = [16, 32, 64]
output_size = 1
complex_model = ComplexNN(input_size, hidden_sizes, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(complex_model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    complex_model.train()
    optimizer.zero_grad()
    outputs = complex_model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

complex_model.eval()
with torch.no_grad():
    complex_val_outputs = complex_model(X_val)
    complex_val_loss = criterion(complex_val_outputs, y_val)

print(f'Complex Model Validation Loss: {complex_val_loss.item():.4f}')

Epoch [100/5000], Loss: 25234384093184.0000
Epoch [200/5000], Loss: 25223258701824.0000
Epoch [300/5000], Loss: 25156781080576.0000
Epoch [400/5000], Loss: 24948431126528.0000
Epoch [500/5000], Loss: 24481370210304.0000
Epoch [600/5000], Loss: 23626566860800.0000
Epoch [700/5000], Loss: 22272003801088.0000
Epoch [800/5000], Loss: 20356563730432.0000
Epoch [900/5000], Loss: 17910368043008.0000
Epoch [1000/5000], Loss: 15079255834624.0000
Epoch [1100/5000], Loss: 12124867788800.0000
Epoch [1200/5000], Loss: 9377140441088.0000
Epoch [1300/5000], Loss: 7127276453888.0000
Epoch [1400/5000], Loss: 5508201185280.0000
Epoch [1500/5000], Loss: 4448932855808.0000
Epoch [1600/5000], Loss: 3769730596864.0000
Epoch [1700/5000], Loss: 3310281818112.0000
Epoch [1800/5000], Loss: 2974572871680.0000
Epoch [1900/5000], Loss: 2715935309824.0000
Epoch [2000/5000], Loss: 2508298387456.0000
Epoch [2100/5000], Loss: 2336111198208.0000
Epoch [2200/5000], Loss: 2189353025536.0000
Epoch [2300/5000], Loss: 20635

In [3]:
#2
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model_a = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_a.parameters(), lr=0.01)
num_epochs = 300
start_time = time.time()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_a(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

end_time = time.time()
training_time_a = end_time - start_time

model_a.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model_a(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_a = correct / total

print(f'Training Time (Part a): {training_time_a} seconds')
print(f'Training Loss (Part a): {running_loss / len(train_loader)}')
print(f'Evaluation Accuracy (Part a): {accuracy_a}')

class ExtendedNN(nn.Module):
    def __init__(self):
        super(ExtendedNN, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model_b = ExtendedNN().to(device)
optimizer = optim.SGD(model_b.parameters(), lr=0.01)

start_time = time.time()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_b(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

end_time = time.time()
training_time_b = end_time - start_time

model_b.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model_b(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_b = correct / total

print(f'Training Time (Part b): {training_time_b} seconds')
print(f'Training Loss (Part b): {running_loss / len(train_loader)}')
print(f'Evaluation Accuracy (Part b): {accuracy_b}')

100%|██████████| 170498071/170498071 [00:01<00:00, 101372154.61it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/300, Loss: 1.8722185852277615
Epoch 2/300, Loss: 1.6552362521286206
Epoch 3/300, Loss: 1.5713234775511504
Epoch 4/300, Loss: 1.5156963580404705
Epoch 5/300, Loss: 1.47000503829678
Epoch 6/300, Loss: 1.4301105181274512
Epoch 7/300, Loss: 1.3935626478451293
Epoch 8/300, Loss: 1.3595741801249706
Epoch 9/300, Loss: 1.327711753985461
Epoch 10/300, Loss: 1.2981395002673655
Epoch 11/300, Loss: 1.271449442486019
Epoch 12/300, Loss: 1.2455610603170322
Epoch 13/300, Loss: 1.2214882939367953
Epoch 14/300, Loss: 1.1970986688838285
Epoch 15/300, Loss: 1.174526668799198
Epoch 16/300, Loss: 1.1516358141246659
Epoch 17/300, Loss: 1.1300545732688416
Epoch 18/300, Loss: 1.1076277956328429
Epoch 19/300, Loss: 1.087171427352959
Epoch 20/300, Loss: 1.0660449570554602
Epoch 21/300, Loss: 1.046731056307283
Epoch 22/300, Loss: 1.0262275868669495
Epoch 23/300, Loss: 1.0058160642986103
Epoch 24/300, Loss: 0.98628643